# Специализция вычислений функций Фибоначчи

## Числи Фибоначчи и числа "Гибоначчи"

In [1]:
fib(n) =
    if n == 0
        0
    elseif n == 1
        1
    else
        fib(n - 2) + fib(n - 1)
    end

fib (generic function with 1 method)

In [2]:
Tuple(fib(n) for n in 0:10)

(0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55)

Проблема в том, что у `fib` - только один аргумент. А если мы хотим специализировать функцию по значениям некоторых аргументов, то этих аргументов должно быть ≥ 2.

Посему, переходим к рассмотрению чисел "Гибоначчи".

In [3]:
gib(n, x, y) =
    if n == 0
        x
    elseif n == 1
        y
    else
        gib(n - 2, x, y) + gib(n - 1, x, y)
    end

gib (generic function with 1 method)

In [4]:
gib(10, 0, 1), gib(10, 2, 3)

(55, 233)

## Мемоизация

In [5]:
function gib_memo!(d, n, x, y)
    if haskey(d, (n, x, y))
        return d[(n, x, y)]
    end

    r =
        if n == 0
            x
        elseif n == 1
            y
        else
            gib_memo!(d, n - 2, x, y) + gib_memo!(d, n - 1, x, y)
        end

    @show n
    d[(n, x, y)] = r
    return r
end

function gib_memo(n, x, y)
    d = Dict{NTuple{3,Int64},Int64}()
    gib_memo!(d, n, x, y)
end

gib_memo (generic function with 1 method)

In [6]:
gib_memo(10, 2, 3)

n = 0
n = 1
n = 2
n = 3
n = 4
n = 5
n = 6
n = 7
n = 8
n = 9
n = 10


233

## Превращаем вычислитель в генератор

Теперь ключом будет только `n`.

In [7]:
include("MacroUtils.jl")
using .MacroUtils: cleanup

In [8]:
function gib_gen!(d, es, n)
    if haskey(d, n)
        return d[n]
    end

    r =
        if n == 0
            :x
        elseif n == 1
            :y
        else
            g = Symbol("g_", n)
            push!(es, :($g = $(gib_gen!(d, es, n - 2)) + $(gib_gen!(d, es, n - 1))))
            g
        end

    # @show n
    d[n] = r
    return r
end

function gib_gen(n)
    d = Dict{Int64,Any}()
    es = Expr[]
    r = gib_gen!(d, es, n)

    quote
        $(es...)
        return $r
    end
end

gib_gen (generic function with 1 method)

In [9]:
gib_gen(0) |> cleanup

:(return x)

In [10]:
gib_gen(1) |> cleanup

:(return y)

In [11]:
gib_gen(3) |> cleanup

quote
    g_2 = x + y
    g_3 = y + g_2
    return g_3
end

In [12]:
gib_gen(10) |> cleanup

quote
    g_2 = x + y
    g_3 = y + g_2
    g_4 = g_2 + g_3
    g_5 = g_3 + g_4
    g_6 = g_4 + g_5
    g_7 = g_5 + g_6
    g_8 = g_6 + g_7
    g_9 = g_7 + g_8
    g_10 = g_8 + g_9
    return g_10
end

In [13]:
@generated function gib_gen(::Val{n}, x, y) where {n}
    gib_gen(n)
end

gib_gen (generic function with 2 methods)

In [14]:
gib_gen(Val(10), 2, 3)

233

## Использование комбинаторов

Потенциальная польза от этого может быть в том, что одно и то же определение функции может исполняться как с мемоизацией, так и без оной.

Или один и тот же механизм мемоизации может использоваться для разных функций.

In [15]:
fib_step(self, n) =
    if n == 0
        0
    elseif n == 1
        1
    else
        self(n - 2) + self(n - 1)
    end

fib_step (generic function with 1 method)

In [16]:
fix(f, x) = f(x1 -> fix(f, x1), x)

fix (generic function with 1 method)

In [17]:
fix(fib_step, 10)

55

А зачем? А затем, что можно добавить а `fix` дополнительные действия, которые будут совершаться перед вызовом функции и после оного.

In [18]:
function fix_trace(f, x)
    println("Calling ", string(f), "(", x, ")")
    r = f(x1 -> fix_trace(f, x1), x)
    println("Returning ", string(f), "(", x, ") = ", r)
    return r
end

fix_trace (generic function with 1 method)

In [19]:
fix_trace(fib_step, 3)

Calling fib_step(3)
Calling fib_step(1)
Returning fib_step(1) = 1
Calling fib_step(2)
Calling fib_step(0)
Returning fib_step(0) = 0
Calling fib_step(1)
Returning fib_step(1) = 1
Returning fib_step(2) = 1
Returning fib_step(3) = 2


2

В частности, не меняя текст `fib_step`, можно подсунуть мемоизацию.

In [20]:
function fix_memo(d, f, x)
    if haskey(d, x)
        d[x]
    else
        r = f(x1 -> fix_memo(d, f, x1), x)
        d[x] = r
        r
    end
end

function fix_memo(f, x)
    d = Dict()
    fix_memo(d, f, x)
end

fix_memo (generic function with 2 methods)

In [21]:
fix_memo(fib_step, 10)

55

## А можно ли определить `fix_trace` и `fix_memo` через стандартный `fix`?

(Непонятно, есть ли в этом практический смысл? Но, с точки зрения науки, - любопытно...)

In [22]:
function trace_step(self, fx)
    f = fx[1]
    x = fx[2]
    println("Calling ", string(f), "(", x, ")")
    r = f(x1 -> self((f, x1)), x)
    println("Returning ", string(f), "(", x, ") = ", r)
    return r
end

trace_step (generic function with 1 method)

In [23]:
fix(trace_step, (fib_step, 3))

Calling fib_step(3)
Calling fib_step(1)
Returning fib_step(1) = 1
Calling fib_step(2)
Calling fib_step(0)
Returning fib_step(0) = 0
Calling fib_step(1)
Returning fib_step(1) = 1
Returning fib_step(2) = 1
Returning fib_step(3) = 2


2

И `fix_memo` можно переопределить через `fix`.

In [24]:
function mk_memo_step()
    d = Dict()

    (self, fx) -> begin
        f = fx[1]
        x = fx[2]

        if haskey(d, x)
            d[x]
        else
            r = f(x1 -> self((f, x1)), x)
            d[x] = r
            r
        end
    end
end

mk_memo_step (generic function with 1 method)

In [25]:
fix(mk_memo_step(), (fib_step, 10))

55

In [26]:
fix2(g, f) = x -> f(x1 -> g(x2 -> fix2(g, f)(x2), x1), x)

fix2 (generic function with 1 method)

In [27]:
fix2(mk_memo_step(), trace_step)((fib_step, 3))

Calling fib_step(3)
Calling fib_step(0)
Returning fib_step(0) = 0
Calling fib_step(1)
Returning fib_step(1) = 1
Returning fib_step(3) = 2


2